In [ ]:
import torch
import lightning as L
import torch.nn.functional as F
import torch.nn as nn
import bentoml

In [ ]:
from __future__ import annotations

import bentoml
from typing import List


bento_image = bentoml.images.Image(python_version="3.11") \
    .python_packages("torch", "transformers")


@bentoml.service(
    image=bento_image,
    resources={"cpu": "4"}
)
class Summarization:
    model_path = bentoml.models.HuggingFaceModel("sshleifer/distilbart-cnn-12-6")

    def __init__(self) -> None:
        import torch
        from transformers import pipeline

        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.pipeline = pipeline('summarization', model=self.model_path, device=device)

    @bentoml.api(batchable=True)
    def summarize(self, texts: List[str]) -> List[str]:
        results = self.pipeline(texts)
        return [item['summary_text'] for item in results]

In [2]:
class Model(L.LightningModule):
    def __init__(self, hidden_size=32,lr=1e-3):
        super().__init__()
        self.save_hyperparameters() 
        self.lr=lr
        self.model=nn.Sequential(nn.Linear(2, hidden_size ),
                                 nn.Tanh(),
                                 nn.Linear(hidden_size, hidden_size),
                                 nn.Tanh(),
                                 nn.Linear(hidden_size,2))
        self.loss_fn = nn.CrossEntropyLoss()
    def forward(self, x):
        return self.model(x)
    
    def predict(self, x):
     
        logits = self(x)
        return F.softmax(logits, dim=1)


In [3]:
model = Model.load_from_checkpoint("best_model.ckpt")


In [7]:
import requests